In [133]:
import pandas as pd
import re

Importing files

In [139]:
fil1 = "../data/ecoli_transcriptomics.xlsx"
data1 = pd.read_excel(fil1)

fil2 = "../data/proteomics_BSY.xlsx"
data2 = pd.read_excel(fil2)

fil3 = "../data/proteomics_ser.xlsx"
data3 = pd.read_excel(fil3)

fil4 = "../data/transcriptomics_ser.xlsx"
data4 = pd.read_excel(fil4)

fil5 = "../data/EC_genome.txt"
data5 = pd.read_csv(fil5, sep='\t')

fil6 = "../data/st_1_variants.txt"
st1_variants = pd.read_csv(fil6, sep='\t')



In [140]:
data5 = data5.drop(data5.index[-1])

Defining function extracting PEG (FigNum) number and adding it to a seperate column

In [141]:
def peg_parser(ori_id):
    test = ori_id.split('"')
    test2 = test[1].split('.')

    if test2[2] == "p eg" :
        gen_id = test2[3]
    elif test2[2] == "r na":
        gen_id = "NaN"

    else:
        print(len(test))
        print(ori_id)

        
    return gen_id

In [142]:
data5['FigNum']=data5['Qualifiers'].map(peg_parser)

In [143]:
data5

,Name,Type,Region,Qualifiers,FigNum
0,mannose-specific adhesin FimH,CDS,complement(2297236..2298150),"/db_xref=""SEED:fig|562.15553.p eg.2181""; /tran...",2181
1,S-adenosylmethionine:tRNA ribosyltransferase-i...,CDS,1148163..1149233,"/db_xref=""SEED:fig|562.15553.p eg.1075""; /db_x...",1075
2,UDP-N-acetylmuramate:L-alanyl-gamma-D-glutamyl...,CDS,573277..574650,"/db_xref=""SEED:fig|562.15553.p eg.524""; /trans...",524
3,LSU ribosomal protein L34p,CDS,607..747,"/db_xref=""SEED:fig|562.15553.p eg.1""; /db_xref...",1
4,Ribonuclease P protein component (EC 3.1.26.5),CDS,797..1123,"/db_xref=""SEED:fig|562.15553.p eg.2""; /db_xref...",2
5,Inner membrane protein translocase component Y...,CDS,1398..2993,"/db_xref=""SEED:fig|562.15553.p eg.3""; /transla...",3
6,GTPase and tRNA-U34 5-formylation enzyme TrmE,CDS,3099..4463,"/db_xref=""SEED:fig|562.15553.p eg.4""; /db_xref...",4
7,Tryptophanase (EC 4.1.99.1),CDS,5001..6416,"/db_xref=""SEED:fig|562.15553.p eg.5""; /db_xref...",5
8,Tryptophan-specific transport protein,CDS,6585..7754,"/db_xref=""SEED:fig|562.15553.p eg.6""; /transla...",6
9,Putative transport protein,CDS,7886..9061,"/db_xref=""SEED:fig|562.15553.p eg.7""; /transla...",7


Making a dictionary to look up the FigNum of corresponding gene to a specific region with mutation

In [144]:
data5_fig_region_dict = {}
for row in range(len(data5)):
    region = data5['Region'][row]
    if "complement" in region:
        region = re.search('\(([^)]+)', region).group(1)
    else:
        pass
    val = list(range(int(region.split('..')[0]), int(region.split('..')[1])+1))
    data5_fig_region_dict[data5['FigNum'][row]]=val

In [145]:
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )

In [146]:
data5_reg_fig_dict = invert(data5_fig_region_dict)

Making a list with the FigNum of the genes affected by mutation

In [147]:
liste = []
for i in range(len(st1_variants)):
    try:
        val = int(st1_variants["Region"][i])
    except ValueError:
        val = st1_variants["Region"][i].split('^')[0]
    if val in data5_reg_fig_dict:
        liste.append(data5_reg_fig_dict[val])

In [148]:
len(liste)

633

In [9]:
data1

,NCBI_GENE_ID,Unnamed: 1,St1_trans,St2_trans,St3_trans,St4_trans,St5_trans,function,evidence_codes,figfam
0,aaeA,fig|1005566.5.peg.3936,3.269841,4.936508,3.523810,4.095238,2.396825,Membrane fusion component of tripartite multid...,"idu(2);Multidrug_Resistance,_Tripartite_System...",FIG00001080
1,aaeB,fig|1005566.5.peg.3937,2.668478,3.940217,2.304348,3.054348,1.967391,FUSARIC ACID RESISTANCE PROTEIN FUSB / FUSARIC...,ff,FIG00634841
2,aaeR,fig|1005566.5.peg.3934,2.029412,3.098039,1.529412,2.289216,2.000000,LysR family transcriptional regulator QseA,idu(1);LysR-family_proteins_in_Escherichia_col...,FIG00001509
3,aas,fig|1005566.5.peg.3525,1.808889,2.456296,1.708148,2.362963,0.986667,2-acylglycerophosphoethanolamine acyltransfera...,icw(2);CBSS-364106.7.peg.3211 icw(3);CBSS-3641...,FIG01953891
4,aat,fig|1005566.5.peg.1551,2.638462,3.023077,1.430769,2.115385,1.461538,Leucyl/phenylalanyl-tRNA--protein transferase ...,isu;Protein_degradation,FIG00000748
5,abgA,fig|1005566.5.peg.2006,5.369048,4.380952,1.345238,2.535714,1.833333,Catalyzes the cleavage of p-aminobenzoyl-gluta...,isu;p-Aminobenzoyl-Glutamate_Utilization,FIG00001384
6,abgB,fig|1005566.5.peg.2005,5.094737,5.347368,1.768421,3.368421,2.168421,Catalyzes the cleavage of p-aminobenzoyl-gluta...,icw(3);p-Aminobenzoyl-Glutamate_Utilization,FIG00139150
7,abgR,fig|1005566.5.peg.2007,1.451524,0.994460,0.407202,0.698061,0.803324,"Regulatory protein (induces abgABT, used to ca...",isu;LysR-family_proteins_in_Escherichia_coli i...,FIG00008656
8,abgT,fig|1005566.5.peg.2004,3.373016,3.500000,1.222222,2.182540,1.658730,Aminobenzoyl-glutamate transport protein,icw(2);p-Aminobenzoyl-Glutamate_Utilization,FIG00010524
9,abrB,fig|1005566.5.peg.1386,0.680180,0.828829,0.382883,0.608108,1.763514,Putative transport protein,ff,FIG00453115


In [16]:
id_test = 'sp|P75780|FIU_ECOLI'

Defining a function extracting gene ID and adding it to a new column

In [11]:
def prot_ID_parser(ori_id):
    test = ori_id.split('|')
    if len(test) >= 3 :
        gen_id = test[2].split('_')[0]
    else:
        gen_id=ori_id
        
    return gen_id[:3].lower()+gen_id[3:]

In [12]:
data2['Gene_ID']=data2['Protein IDs'].map(prot_ID_parser)

In [13]:
data2

,Protein IDs,St7_prot,St8_prot,St9_prot,Gene_ID
0,sp|P75780|FIU_ECOLI,NaN,NaN,NaN,fiu
1,sp|P14175|PROV_ECOLI,NaN,NaN,NaN,proV
2,sp|P77667|SUFA_ECOLI,NaN,NaN,NaN,sufA
3,sp|P06960|OTC2_ECOLI,0.000000,1.800000,1.000000,otc2
4,CON__P13645,0.000000,0.750000,0.333333,con__P13645
5,CON__P07477,0.000000,6.000000,2.000000,con__P07477
6,sp|P64545|YFGG_ECOLI,0.000000,9.000000,4.000000,yfgG
7,sp|P31828|PQQL_ECOLI,0.000000,6.000000,3.000000,pqqL
8,sp|P42632|TDCE_ECOLI,NaN,NaN,NaN,tdcE
9,sp|P76272|YEBT_ECOLI,0.000000,0.750000,0.750000,yebT


Merging transcriptomic data for strain 1-5, 7-9)

In [30]:
data1.merge(data2,how = 'left' , left_on="NCBI_GENE_ID", right_on = "Gene_ID")

,NCBI_GENE_ID,Unnamed: 1,St1_trans,St2_trans,St3_trans,St4_trans,St5_trans,function,evidence_codes,figfam,Protein IDs,St7_prot,St8_prot,St9_prot,Gene_ID
0,aaeA,fig|1005566.5.peg.3936,3.269841,4.936508,3.523810,4.095238,2.396825,Membrane fusion component of tripartite multid...,"idu(2);Multidrug_Resistance,_Tripartite_System...",FIG00001080,NaN,NaN,NaN,NaN,NaN
1,aaeB,fig|1005566.5.peg.3937,2.668478,3.940217,2.304348,3.054348,1.967391,FUSARIC ACID RESISTANCE PROTEIN FUSB / FUSARIC...,ff,FIG00634841,NaN,NaN,NaN,NaN,NaN
2,aaeR,fig|1005566.5.peg.3934,2.029412,3.098039,1.529412,2.289216,2.000000,LysR family transcriptional regulator QseA,idu(1);LysR-family_proteins_in_Escherichia_col...,FIG00001509,NaN,NaN,NaN,NaN,NaN
3,aas,fig|1005566.5.peg.3525,1.808889,2.456296,1.708148,2.362963,0.986667,2-acylglycerophosphoethanolamine acyltransfera...,icw(2);CBSS-364106.7.peg.3211 icw(3);CBSS-3641...,FIG01953891,NaN,NaN,NaN,NaN,NaN
4,aat,fig|1005566.5.peg.1551,2.638462,3.023077,1.430769,2.115385,1.461538,Leucyl/phenylalanyl-tRNA--protein transferase ...,isu;Protein_degradation,FIG00000748,sp|P00509|AAT_ECOLI,1.126984,3.238095,1.174603,aat
5,abgA,fig|1005566.5.peg.2006,5.369048,4.380952,1.345238,2.535714,1.833333,Catalyzes the cleavage of p-aminobenzoyl-gluta...,isu;p-Aminobenzoyl-Glutamate_Utilization,FIG00001384,NaN,NaN,NaN,NaN,NaN
6,abgB,fig|1005566.5.peg.2005,5.094737,5.347368,1.768421,3.368421,2.168421,Catalyzes the cleavage of p-aminobenzoyl-gluta...,icw(3);p-Aminobenzoyl-Glutamate_Utilization,FIG00139150,NaN,NaN,NaN,NaN,NaN
7,abgR,fig|1005566.5.peg.2007,1.451524,0.994460,0.407202,0.698061,0.803324,"Regulatory protein (induces abgABT, used to ca...",isu;LysR-family_proteins_in_Escherichia_coli i...,FIG00008656,NaN,NaN,NaN,NaN,NaN
8,abgT,fig|1005566.5.peg.2004,3.373016,3.500000,1.222222,2.182540,1.658730,Aminobenzoyl-glutamate transport protein,icw(2);p-Aminobenzoyl-Glutamate_Utilization,FIG00010524,NaN,NaN,NaN,NaN,NaN
9,abrB,fig|1005566.5.peg.1386,0.680180,0.828829,0.382883,0.608108,1.763514,Putative transport protein,ff,FIG00453115,NaN,NaN,NaN,NaN,NaN


In [18]:
len(data1)

4522

In [31]:
len(data2)

1616